# Shark Attacks

Objetivos:

Identificar se fatalidades estão relacionadas a alguma espécie específica;

Identificar se ataques ocorrem em uma época específica e porque.

In [525]:
import os
import pandas as pd
import numpy as np

In [526]:
shark_data = pd.read_csv("attacks.csv")

## Dados originais

In [527]:
shark_data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [528]:
shark_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

## Retirada de colunas que não serão utilizadas e limpeza das restantes

In [529]:
shark_data.drop('Name', axis=1)
cshark_data = shark_data.drop(['Name', 'original order', 'Investigator or Source', 'pdf', 'href formula', 'Case Number.1','Case Number.2', 'Unnamed: 22', 'Unnamed: 23'], axis=1)

In [530]:
shark_columns = list(cshark_data.columns)
shark_clean_col = [col.lower().strip().replace(' ','_') for col in shark_columns]

In [531]:
cshark_data.columns = shark_clean_col
cshark_data.head(50)

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_(y/n),time,species,href
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",http://sharkattackfile.net/spreadsheets/pdf_di...
5,2018.06.03.b,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,M,NaN,"No injury, board bitten",N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
6,2018.06.03.a,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,M,18,FATAL,Y,Late afternoon,Tiger shark,http://sharkattackfile.net/spreadsheets/pdf_di...
7,2018.05.27,27-May-2018,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'",http://sharkattackfile.net/spreadsheets/pdf_di...
8,2018.05.26.b,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'",http://sharkattackfile.net/spreadsheets/pdf_di...
9,2018.05.26.a,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,M,12,Minor injury to foot,N,14h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...


In [532]:
cshark_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   case_number  8702 non-null   object 
 1   date         6302 non-null   object 
 2   year         6300 non-null   float64
 3   type         6298 non-null   object 
 4   country      6252 non-null   object 
 5   area         5847 non-null   object 
 6   location     5762 non-null   object 
 7   activity     5758 non-null   object 
 8   sex          5737 non-null   object 
 9   age          3471 non-null   object 
 10  injury       6274 non-null   object 
 11  fatal_(y/n)  5763 non-null   object 
 12  time         2948 non-null   object 
 13  species      3464 non-null   object 
 14  href         6302 non-null   object 
dtypes: float64(1), object(14)
memory usage: 2.9+ MB


## Primeira análise dos dados

In [533]:
cshark_data.describe()

,year
count,6300.000000
mean,1927.272381
std,281.116308
min,0.000000
25%,1942.000000
50%,1977.000000
75%,2005.000000
max,2018.000000


In [534]:
#A a maior parte nos estados unidos, surfando, homens, 17 anos, perto de 2015,
#verificar injury pois não está coerente, a tarde, tubarões brancos.

## Identificação dos dados unicos de cada coluna e verificar o que precisa ser limpo

In [535]:
cshark_data['type'].unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [536]:
cshark_data['type'].value_counts()

Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: type, dtype: int64

In [537]:
cshark_data['country'].unique()

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA',
       'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA',
       'ECUADOR', 'MALAYSIA', 'LIBYA', nan, 'CUBA', 'MAURITIUS',
       'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN',
       'EGYPT', 'ST HELENA, British overseas territory', 'COMOROS',
       'REUNION', 'FRENCH POLYNESIA', 'UNITED KINGDOM',
       'UNITED ARAB EMIRATES', 'PHILIPPINES', 'INDONESIA', 'CHINA',
       'COLUMBIA', 'CAPE VERDE', 'Fiji', 'DOMINICAN REPUBLIC',
       'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE', 'FIJI', 'PUERTO RICO',
       'ITALY', 'ATLANTIC OCEAN', 'GREECE', 'ST. MARTIN', 'FRANCE',
       'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL',
       'DIEGO GARCIA', 'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES',
       'GUAM', 'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND',
       'CANADA', 'CROATIA', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA',
       'RUSSIA', 'TURKS & CAICOS', 'UNITE

In [538]:
cshark_data['country'].value_counts()

USA                             2229
AUSTRALIA                       1338
SOUTH AFRICA                     579
PAPUA NEW GUINEA                 134
NEW ZEALAND                      128
                                ... 
GREENLAND                          1
EQUATORIAL GUINEA / CAMEROON       1
RED SEA?                           1
MID-PACIFC OCEAN                   1
WESTERN SAMOA                      1
Name: country, Length: 212, dtype: int64

In [539]:
cshark_data['age'].unique()

array(['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21',
       '34', '30', '60', '33', '29', '54', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8',
       '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen',
       '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6',
       '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53',
       '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20',
       '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26',
       '8 or 10', '84', '� ', ' ', '30 or 36', '6�', '21 & ?', '75',
       '33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28',
       '20?', "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly',
       'mid-20s', 'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50',
       '18 to 22', 'adult', '9 & 12', '? & 19', '9 months', '25 to 35',
     

In [540]:
cshark_data['age'].value_counts()

17          154
18          150
19          142
20          141
15          139
           ... 
46 & 34       1
20            1
16 to 18      1
25 to 35      1
28 & 26       1
Name: age, Length: 157, dtype: int64

In [541]:
cshark_data['activity'].unique()

array(['Paddling', 'Standing', 'Surfing', ...,
       'Crew swimming alongside their anchored ship',
       '4 men were bathing', 'Wreck of  large double sailing canoe'],
      dtype=object)

In [542]:
cshark_data['activity'].value_counts()

Surfing                                                                                                                                                                                               971
Swimming                                                                                                                                                                                              869
Fishing                                                                                                                                                                                               431
Spearfishing                                                                                                                                                                                          333
Bathing                                                                                                                                                                                         

In [543]:
cshark_data['species'].unique()

array(['White shark', nan, '2 m shark', ..., "12' tiger shark",
       'Blue pointers',
       'Said to involve a grey nurse shark that leapt out of the water and  seized the boy but species identification is questionable'],
      dtype=object)

In [544]:
cshark_data['species'].value_counts()

White shark                                           163
Shark involvement prior to death was not confirmed    105
Invalid                                               102
Shark involvement not confirmed                        88
Tiger shark                                            73
                                                     ... 
5' to 8' shark                                          1
3.3 m [10'9"] shark                                     1
Blacktip reef shark pup                                 1
White shark, 12' to 15'                                 1
Nurse shark, 1.1 m [3.5']                               1
Name: species, Length: 1549, dtype: int64

In [545]:
cshark_data['year'].unique()

array([2018., 2017.,   nan, 2016., 2015., 2014., 2013., 2012., 2011.,
       2010., 2009., 2008., 2007., 2006., 2005., 2004., 2003., 2002.,
       2001., 2000., 1999., 1998., 1997., 1996., 1995., 1984., 1994.,
       1993., 1992., 1991., 1990., 1989., 1969., 1988., 1987., 1986.,
       1985., 1983., 1982., 1981., 1980., 1979., 1978., 1977., 1976.,
       1975., 1974., 1973., 1972., 1971., 1970., 1968., 1967., 1966.,
       1965., 1964., 1963., 1962., 1961., 1960., 1959., 1958., 1957.,
       1956., 1955., 1954., 1953., 1952., 1951., 1950., 1949., 1948.,
       1848., 1947., 1946., 1945., 1944., 1943., 1942., 1941., 1940.,
       1939., 1938., 1937., 1936., 1935., 1934., 1933., 1932., 1931.,
       1930., 1929., 1928., 1927., 1926., 1925., 1924., 1923., 1922.,
       1921., 1920., 1919., 1918., 1917., 1916., 1915., 1914., 1913.,
       1912., 1911., 1910., 1909., 1908., 1907., 1906., 1905., 1904.,
       1903., 1902., 1901., 1900., 1899., 1898., 1897., 1896., 1895.,
       1894., 1893.,

In [546]:
cshark_data['year'].value_counts()

2015.0    143
2017.0    136
2016.0    130
2011.0    128
2014.0    127
         ... 
1785.0      1
1580.0      1
1742.0      1
1818.0      1
1786.0      1
Name: year, Length: 249, dtype: int64

# Limpeza da tabela

Há muitas linhas sem valor nenhum. Retirar as linhas com muitos NaN

In [547]:
cshark_data.tail(100)

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_(y/n),time,species,href
25623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [548]:
cshark = cshark_data.dropna(thresh = 0.1*len(cshark_data.columns), axis=0)

## Limpeza das datas

Limpar as datas para aplicação de .to_datetime 

In [549]:
import re

In [550]:
 re.findall('\d{2}-\D{3}-\d+', "Before 13-jun-1998") #Teste de Regex

['13-jun-1998']

In [551]:
def change_date(name):
    date1 = re.findall('\d{2}-\D{3}-\d+', name)
    if len(date1)>0:
        return date1[0]

In [552]:
cshark['date'] = cshark['date'].apply(change_date)

<ipython-input-552-803a69df7c4a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cshark['date'] = cshark['date'].apply(change_date)


In [582]:
cshark['date'] = pd.to_datetime(cshark['date'])

In [583]:
cshark.info() #Verificação do formato da coluna Date

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6300 entries, 0 to 6301
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         5348 non-null   datetime64[ns]
 1   year         6298 non-null   object        
 2   type         6296 non-null   object        
 3   country      6250 non-null   object        
 4   area         5846 non-null   object        
 5   location     5761 non-null   object        
 6   activity     5756 non-null   object        
 7   injury       6272 non-null   object        
 8   fatal_(y/n)  5761 non-null   object        
 9   time         2947 non-null   object        
 10  species      3463 non-null   object        
dtypes: datetime64[ns](1), object(10)
memory usage: 719.7+ KB


In [554]:
cshark['date'][4600:4650]


4600    19-Apr-1946
4601           None
4602    20-Mar-1946
4603    16-Feb-1946
4604    10-Feb-1946
4605    10-Feb-1946
4606    24-Jan-1946
4607    24-Jan-1946
4608    17-Jan-1946
4609    07-Jan-1946
4610    05-Jan-1946
4611    01-Jan-1946
4612           None
4613           None
4614           None
4615    23-Sep-1945
4616    19-Sep-1945
4617    16-Sep-1945
4618    06-Sep-1945
4619    19-Aug-1945
4620    06-Aug-1945
4621    30-Jul-1945
4622           None
4623    15-Jun-1945
4624    08-May-1945
4625    06-Mar-1945
4626    05-Feb-1945
4627           None
4628           None
4629           None
4630           None
4631           None
4632    02-Dec-1944
4633    08-Nov-1944
4634    01-Nov-1944
4635    26-Oct-1944
4636    26-Oct-1944
4637    25-Oct-1944
4638    25-Oct-1944
4639    24-Oct-1944
4640    23-Oct-1944
4641    03-Sep-1944
4642           None
4643    20-Aug-1944
4644     22-Jul-144
4645    23-Jun-1944
4646    31-May-1944
4647    24-May-1944
4648    04-May-1944
4649           None


In [581]:
cshark=cshark.drop(index=[4644])

KeyError: '[4644] not found in axis'

In [580]:
cshark=cshark.drop(index=[6170])

## Limpeza das espécies

In [556]:
cshark['species'].value_counts().head(50)

White shark                                           163
Shark involvement prior to death was not confirmed    105
Invalid                                               102
Shark involvement not confirmed                        88
Tiger shark                                            73
Shark involvement prior to death unconfirmed           68
Bull shark                                             52
4' shark                                               40
6' shark                                               40
1.8 m [6'] shark                                       35
Questionable incident                                  35
Questionable                                           34
1.5 m [5'] shark                                       32
1.2 m [4'] shark                                       27
5' shark                                               26
3' shark                                               26
2 m shark                                              25
4' to 5' shark

In [557]:
cshark[cshark['species']=="Invalid"].head(30)

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_(y/n),time,species,href
226,2016.09.00,None,2016.0,Invalid,MEXICO,NaN,Guadalupe Island,Cage Diving,NaN,NaN,"No injury to divers, white shark breached cage",NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
578,2014.03.13,13-Mar-2014,2014.0,Invalid,CAYMAN ISLANDS,NaN,NaN,Scuba diving / culling lionfish,M,NaN,"Caribbean reef shark buzzed him. No injury, no...",NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
676,2013.05.08.b,08-May-2013,2013.0,Invalid,USA,California,"Tourmaline Surf Park, San Diego County",Surfing,M,42,Shark bites were post-mortem,NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
926,2011.05.07.R,07-May-2011,2011.0,Invalid,UNITED ARAB EMIRATES (UAE),Umm al Qaywayan Province,Khor Fakkan,Fishing,M,43,Erroneously reported on several internet sites...,NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
1044,2010.02.04,04-Feb-2010,2010.0,Invalid,GUAM,Merizo,Achang Reef,Spearfishing (free diving),M,31,Shark bites were post-mortem,NaN,11h00,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
1093,2009.09.02,02-Sep-2009,2009.0,Invalid,NEVIS,NaN,Castle Beach,Swimming,M,NaN,Death was due to drowning. Two days later his ...,NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
1167,2009.01.23,23-Jan-2009,2009.0,Invalid,BRAZIL,Maranh�o,Olho d'�gua,Swimming,M,17,"Drowned, body scavenged by shark",NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
1301,2007.12.19,19-Dec-2007,2007.0,Invalid,BRITISH VIRGIN ISLANDS,Green Bay,NaN,Scuba diving,M,53,Shark bites were post-mortem,NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
1314,2007.11.00,None,2007.0,Invalid,MEXICO,Baja California,Guadalupe Island,Shark diving,M,NaN,White shark breached cage. No injury to occupants,NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
1427,2006.09.30,30-Sep-2006,2006.0,Invalid,SOUTH AFRICA,Western Cape Province,Miller's Point,Spearfishing,M,36,No injury; 4m white shark made a threat display,NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...


In [250]:
re.sub('.* ?[Ww]hite [Ss]hark.*', 'WhiteShark','a huge white shark said hello and invited the man to dinner.')

'WhiteShark'

In [273]:
re.sub('.* [Ss]hark.*', 'puppy','WhiteShark ')

'WhiteShark '

In [271]:
re.sub('.* ?[Ss]hark.*', 'puppy','White Shark ')

'puppy'

In [558]:
shark_dict1 = {'.* ?[Qq]uestionable ?.*': 'Invalid',
               '.* ?[Nn]ot [Cc]onfirmed ?.*': 'Invalid',
               '.* ?[Ww]hite [Ss]hark ?.*': 'WhiteShark', 
               '.* ?[Bb]ull [Ss]hark ?.*': 'BullShark', 
               '.* ?[Tt]iger [Ss]hark ?.*': 'TigerShark', 
               '.* ?[Nn]urse [Ss]hark ?.*': 'NurseShark',
               '.* ?[Nn]o [Ss]hark ?.*': 'Invalid',
               '.* ?[Hh]ammerhead [Ss]hark ?.*': 'HammerheadShark',
               '.* ?[Bb]ronze ?[Ww]haler [Ss]hark ?.*': 'BronzewhalerShark',
               '.* ?[Ww]haler [Ss]hark ?.*': 'WhalerShark',
               '.* ?[Ww]obbegong[ s][,]?  [Ss]hark ?.*': 'WobbegongShark',
               '.* ?[Ww]obbegong[,]? ?.*': 'WobbegongShark',
               '.* ?[Bb]lacktip [Ss]hark ?.*': 'BlacktipShark',
               '.* ?[Ll]emon [Ss]hark ?.*': 'LemonShark',
               '.* ?[Bb]lue [Ss]hark ?.*': 'BlueShark',
               '.* ?[Ww]hitetip [Ss]hark ?.*': 'WhitetipShark',
               '.* ?[Gg]r[ae]y [Rr]eef [Ss]hark ?.*': 'GreyreefShark',
               '.* ?[Bb]lue [Pp]ointers ?.*': 'BluePointers'}

In [559]:
def change_spec1(name,shark_dict1,key):
    if isinstance(name,str):
        return re.sub(key, shark_dict1[key], name)
    else:
        return None

In [561]:
for key in shark_dict1.keys():
    cshark['species'] = cshark['species'].apply(change_spec1, shark_dict1=shark_dict1, key=key)

In [562]:
shark_dict2 = {'.* [Ss]hark ?.*': 'UnknownShark',
               '.* [Dd]rowining ?.*': 'Invalid',
               '.* [Uu]nconfirmed ?.*': 'Invalid',
               '.* [Tt]o [Bb]e [Cc]onfirmed ?.*': 'Invalid',
               '.* ?\d+ m .*': 'UnknownShark',
               ".* ?\d+ cm .*": 'UnknownShark',
               '.* ?[Ii]nvalid ?.*': 'Invalid'}

In [563]:
def change_unknown(name,shark_dict2,key):
    if isinstance(name,str):
        return re.sub(key, shark_dict2[key], name)
    else:
        return None

In [564]:
for key in shark_dict2.keys():
    cshark['species'] = cshark['species'].apply(change_unknown, shark_dict2=shark_dict2, key=key)

In [565]:
cshark['species'].value_counts()

UnknownShark                                                                                                                                                     1359
WhiteShark                                                                                                                                                        632
Invalid                                                                                                                                                           506
TigerShark                                                                                                                                                        281
BullShark                                                                                                                                                         180
NurseShark                                                                                                                                                         96
Blac

In [566]:
cshark

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_(y/n),time,species,href
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,WhiteShark,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,Minor injury to left thigh,N,14h00 -15h00,None,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,48,Injury to left lower leg from surfboard skeg,N,07h45,None,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,NaN,Minor injury to lower leg,N,NaN,UnknownShark,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,TigerShark,http://sharkattackfile.net/spreadsheets/pdf_di...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,None,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,M,NaN,FATAL,Y,NaN,None,http://sharkattackfile.net/spreadsheets/pdf_di...
6298,ND.0004,None,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,M,NaN,FATAL,Y,NaN,None,http://sharkattackfile.net/spreadsheets/pdf_di...
6299,ND.0003,None,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,M,NaN,FATAL,Y,NaN,None,http://sharkattackfile.net/spreadsheets/pdf_di...
6300,ND.0002,None,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8�N, 79�W",NaN,M,NaN,FATAL,Y,NaN,None,http://sharkattackfile.net/spreadsheets/pdf_di...


In [567]:
cshark = cshark.drop(['sex', 'age', 'case_number', 'href'], axis=1)

In [568]:
cshark

,date,year,type,country,area,location,activity,injury,fatal_(y/n),time,species
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,"No injury to occupant, outrigger canoe and pad...",N,18h00,WhiteShark
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Minor injury to left thigh,N,14h00 -15h00,None
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,Injury to left lower leg from surfboard skeg,N,07h45,None
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,Minor injury to lower leg,N,NaN,UnknownShark
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,TigerShark
...,...,...,...,...,...,...,...,...,...,...,...
6297,None,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,FATAL,Y,NaN,None
6298,None,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,FATAL,Y,NaN,None
6299,None,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,FATAL,Y,NaN,None
6300,None,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8�N, 79�W",NaN,FATAL,Y,NaN,None


## Limpeza da coluna fatal

A coluna injury e a coluna fatal não apresentam dados coerentes entre elas. Verificar.

In [571]:
shark_dict3 = {' ?[Nn] ?': 'N',
               ' ?[Yy] ?': 'Y',
               ' ?[Mm] ?': 'N',
               ' ?UNKNOWN ?': 'UNKNOWN',
               '\d+': 'UNKNOWN'}

In [572]:
def change_inj(name,shark_dict3,key):
    if isinstance(name,str):
        return re.sub(key, shark_dict3[key], name)
    else:
        return None

In [573]:
for key in shark_dict3.keys():
    cshark['fatal_(y/n)'] = cshark['fatal_(y/n)'].apply(change_inj, shark_dict3=shark_dict3, key=key)

In [574]:
cshark['fatal_(y/n)'].value_counts()

N          4302
Y          1388
UNKNOWN      72
Name: fatal_(y/n), dtype: int64

## Limpeza Locais

In [575]:
cshark['area'].value_counts()

Florida                              1037
New South Wales                       486
Queensland                            311
Hawaii                                298
California                            290
                                     ... 
Cavite Province, Luzon                  1
Island of Volos                         1
Makora-Ulawa Province                   1
Phang nga Province                      1
Between New Ireland & New Britain       1
Name: area, Length: 825, dtype: int64

In [576]:
cshark['year'] = cshark['year'].astype(object)

In [577]:
pd.options.display.max_rows=100

In [499]:
cshark.groupby(['fatal_(y/n)', 'species'], as_index=False).count().sort_values(['fatal_(y/n)','year'], ascending = False)

,fatal_(y/n),species,date,year,type,country,area,location,activity,injury,time
69,Y,WhiteShark,136,143,143,143,140,141,139,143,113
67,Y,UnknownShark,114,137,137,135,126,130,128,137,63
65,Y,TigerShark,57,69,69,69,64,64,66,69,41
58,Y,BullShark,34,40,40,40,36,37,35,40,24
61,Y,Invalid,6,9,9,8,8,9,9,9,3
56,Y,BlueShark,7,7,7,6,5,5,5,7,6
70,Y,WhitetipShark,5,5,5,5,5,5,5,5,3
57,Y,BronzewhalerShark,4,4,4,4,4,4,4,4,4
54,Y,Blue pointer,0,2,2,2,0,0,2,2,0
59,Y,HammerheadShark,1,2,2,2,2,1,2,2,0


In [584]:
cshark['month'] = cshark['date'].dt.month

In [585]:
cshark['month'].value_counts()

7.0     611
8.0     546
9.0     519
1.0     491
6.0     470
10.0    417
4.0     415
12.0    413
3.0     378
11.0    377
5.0     357
2.0     354
Name: month, dtype: int64

In [586]:
pd.options.display.max_rows=100
cshark.groupby(['month', 'species', 'area']).count()

date  year  type  country  \
month species        area                                               
1.0                  Pernambuco                1     1     1        1   
      BlacktipShark  Florida                   1     1     1        1   
                     KwaZulu-Natal             3     3     3        3   
                     Queensland                1     1     1        1   
      Blue pointer   South Australia           1     1     1        1   
...                                          ...   ...   ...      ...   
12.0  WhiteShark     Western Australia         3     3     3        3   
                     Western Cape Province    10    10    10       10   
      WhitetipShark  South Sinai Peninsula     1     1     1        1   
      WobbegongShark New South Wales           6     6     6        6   
                     Queensland                1     1     1        1   

                                            location  activity  injury  \
month species        area                                                
1.0                  Pernambuco                    1         1       1   
      BlacktipShark  Florida                       1         1       1   
                     KwaZulu-Natal                 3         3       3   
                     Queensland                    1         1       1   
      Blue pointer   South Australia               1         0       1   
...                                              ...       ...     ...   
12.0  WhiteShark     Western Australia             3         3       3   
                     Western Cape Province        10        10      10   
      WhitetipShark  South Sinai Peninsula         1         1       1   
      WobbegongShark New South Wales               6         6       6   
                     Queensland                    1         1       1   

                                            fatal_(y/n)  time  
month species        area                                      
1.0                  Pernambuco                       1     0  
      BlacktipShark  Florida                          1     1  
                     KwaZulu-Natal                    3     2  
                     Queensland                       1     1  
      Blue pointer   South Australia                  1     0  
...                                                 ...   ...  
12.0  WhiteShark     Western Australia                3     3  
                     Western Cape Province           10    10  
      WhitetipShark  South Sinai Peninsula            1     1  
      WobbegongShark New South Wales                  6     2  
                     Queensland                       1     0  

[1332 rows x 9 columns]

In [588]:
cshark2=cshark.copy(deep=True)

In [596]:
cshark2

,date,year,type,country,area,location,activity,injury,fatal_(y/n),time,species,month
0,2018-06-25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,"No injury to occupant, outrigger canoe and pad...",N,18h00,WhiteShark,6.0
1,2018-06-18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Minor injury to left thigh,N,14h00 -15h00,None,6.0
2,2018-06-09,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,Injury to left lower leg from surfboard skeg,N,07h45,None,6.0
3,2018-06-08,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,Minor injury to lower leg,N,NaN,UnknownShark,6.0
4,2018-06-04,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,TigerShark,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6297,NaT,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,FATAL,Y,NaN,None,NaN
6298,NaT,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,FATAL,Y,NaN,None,NaN
6299,NaT,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,FATAL,Y,NaN,None,NaN
6300,NaT,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8�N, 79�W",NaN,FATAL,Y,NaN,None,NaN


In [598]:
mask = ((cshark2['species']=="WhiteShark") | (cshark2['species']=="Invalid") | (cshark2['species']=="BullShark") | (cshark2['species']=="UnknownShark") | (cshark2['species']=="TigerShark"))

In [600]:
cshark2=cshark2[mask]

In [601]:
cshark2

,date,year,type,country,area,location,activity,injury,fatal_(y/n),time,species,month
0,2018-06-25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,"No injury to occupant, outrigger canoe and pad...",N,18h00,WhiteShark,6.0
3,2018-06-08,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,Minor injury to lower leg,N,NaN,UnknownShark,6.0
4,2018-06-04,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,TigerShark,6.0
6,2018-06-03,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,FATAL,Y,Late afternoon,TigerShark,6.0
8,2018-05-26,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Lower left leg bitten,N,17h00,BullShark,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6266,NaT,0.0,Unprovoked,SAUDI ARABIA,Eastern Province,East of the Ras Tanura-Jubail area,Diving,"FATAL, died of sepsis",Y,NaN,UnknownShark,NaN
6267,NaT,0.0,Unprovoked,SAUDI ARABIA,Eastern Province,East of the Ras Tanura-Jubail area,Diving,Buttocks bitten,N,NaN,UnknownShark,NaN
6274,NaT,0.0,Boat,ITALY,Adriatic Sea,NaN,Wooden fishing boat,No injury to occupant; shark capsized boat,N,NaN,WhiteShark,NaN
6276,NaT,0.0,Unprovoked,BELIZE,NaN,NaN,Standing,FATAL,Y,NaN,TigerShark,NaN


In [603]:
cshark2.groupby(['fatal_(y/n)', 'species'], as_index=False).count().sort_values(['fatal_(y/n)','year'], ascending = False)

,fatal_(y/n),species,date,year,type,country,area,location,activity,injury,time,month
13,Y,WhiteShark,136,143,143,143,140,141,139,143,113,136
12,Y,UnknownShark,114,137,137,135,126,130,128,137,63,114
11,Y,TigerShark,57,69,69,69,64,64,66,69,41,57
9,Y,BullShark,34,40,40,40,36,37,35,40,24,34
10,Y,Invalid,6,9,9,8,8,9,9,9,3,6
8,UNKNOWN,WhiteShark,3,7,7,7,7,6,6,5,0,3
7,UNKNOWN,UnknownShark,2,3,3,3,3,3,3,3,0,2
5,UNKNOWN,Invalid,2,2,2,2,2,2,2,1,2,2
6,UNKNOWN,TigerShark,1,2,2,2,2,2,0,1,0,1
3,N,UnknownShark,1092,1204,1204,1202,1168,1161,1166,1204,690,1092


# Criando mascaras separadas

In [642]:
mask_white = cshark2['species']=="WhiteShark"
mask_inv = cshark2['species']=="Invalid"
mask_unk = cshark2['species']=="UnknownShark"
mask_bull = cshark2['species']=="BullShark"

In [643]:
white_shark=cshark2[mask_white]
invalid_att=cshark2[mask_inv]
unknown_shark=cshark2[mask_unk]
bull_shark=cshark2[mask_bull]

In [644]:
unknown_shark

,date,year,type,country,area,location,activity,injury,fatal_(y/n),time,species,month
3,2018-06-08,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,Minor injury to lower leg,N,NaN,UnknownShark,6.0
21,2018-04-25,2018.0,Unprovoked,AUSTRALIA,Western Australia,"Surfers Point, Prevelly",Surfing,"No injury, knocked off board by shark",N,09h30,UnknownShark,4.0
25,2018-04-19,2018.0,Unprovoked,USA,Hawaii,"Shipwreck�s Beach, Keoneloa Bay, Kauai",Body boarding,Lacerations to right lower leg,N,NaN,UnknownShark,4.0
32,2018-04-09,2018.0,Unprovoked,NEW CALEDONIA,NaN,"Magenta Beach, Noumea",Windsurfing,"No injury, shark bit board",N,17h00,UnknownShark,4.0
37,NaT,2018.0,Unprovoked,AUSTRALIA,Victoria,Lorne,Swimming,"No injury, said to have been charged by sharks...",N,15h30,UnknownShark,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6238,NaT,0.0,Unprovoked,FIJI,NaN,NaN,Spearfishing,"No injury, bumped by shark which took speared ...",N,NaN,UnknownShark,NaN
6239,NaT,0.0,Unprovoked,MOZAMBIQUE,Maputo Province,Santa Maria Peninsula,Skindiving,Bumped by sharks,N,NaN,UnknownShark,NaN
6259,NaT,0.0,Sea Disaster,NaN,NaN,NaN,A group of survivors on a raft for 17-days,"FATAL, shark leapt into raft and bit the man w...",Y,Late afternoon,UnknownShark,NaN
6266,NaT,0.0,Unprovoked,SAUDI ARABIA,Eastern Province,East of the Ras Tanura-Jubail area,Diving,"FATAL, died of sepsis",Y,NaN,UnknownShark,NaN


# White Shark

In [628]:
white_shark['fatal_(y/n)'].value_counts()

N          477
Y          143
UNKNOWN      7
Name: fatal_(y/n), dtype: int64

In [660]:
white_shark['area'].value_counts()

California                               156
Western Cape Province                    102
South Australia                           41
Western Australia                         41
New South Wales                           38
Eastern Cape Province                     36
KwaZulu-Natal                             23
Oregon                                    22
Victoria                                  12
Tyrrhenian Sea                            10
Tasmania                                   9
Adriatic Sea                               8
Queensland                                 7
South Island                               6
North Carolina                             5
North Island                               5
Florida                                    4
Sicily                                     4
Massachusetts                              4
Ehime Prefecture                           3
New Jersey                                 3
Corfu Island                               3
Hawaii    

In [661]:
maskw = ((white_shark['area']=="California") | (cshark2['area']=="Western Cape Province") | (cshark2['area']=="South Australia ") | (cshark2['area']=="Western Australia") | (cshark2['area']=="New South Wales"))

In [663]:
white_shark2=white_shark[maskw]

<ipython-input-663-927814ba8beb>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  white_shark2=white_shark[maskw]


In [667]:
white_shark2.groupby(['area', 'month'], as_index=False).count().sort_values(['year'], ascending = False)

,area,month,date,year,type,country,location,activity,injury,fatal_(y/n),time,species
7,California,8.0,27,27,27,27,27,27,27,27,25,27
8,California,9.0,25,25,25,25,25,25,25,25,22,25
6,California,7.0,21,21,21,21,21,21,21,21,14,21
9,California,10.0,21,21,21,21,21,21,21,21,20,21
10,California,11.0,14,14,14,14,14,13,14,14,9,14
40,Western Cape Province,6.0,11,11,11,11,11,11,11,11,9,11
43,Western Cape Province,9.0,11,11,11,11,11,11,11,11,9,11
46,Western Cape Province,12.0,10,10,10,10,10,10,10,10,10,10
5,California,6.0,10,10,10,10,10,9,10,10,8,10
44,Western Cape Province,10.0,9,9,9,9,9,9,9,9,5,9


# Bull Shark

In [631]:
bull_shark['fatal_(y/n)'].value_counts()

N    139
Y     40
Name: fatal_(y/n), dtype: int64

In [632]:
bull_shark['area'].value_counts()

Florida                         46
Queensland                      14
New South Wales                 13
Abaco Islands                    8
North Carolina                   8
Karun River                      7
Texas                            6
Pernambuco                       5
South Carolina                   4
Basrah                           4
Alabama                          3
Bali                             3
North Province                   3
Western Australia                3
Guantanamo Province              2
Lake Nicaragua (fresh water)     2
Louisiana                        2
Grand Bahama Island              2
Eastern Cape Province            2
Anzoategui                       2
Northern Bahamas                 1
Vava'u                           1
Sainte-Marie                     1
Basrah City                      1
Saint-Paul                       1
Saint Leu                        1
US Virgin Islands                1
Camaguey Province                1
Praslin             

In [666]:
bull_shark.groupby(['area', 'month'], as_index=False).count().sort_values(['year'], ascending = False)

,area,month,date,year,type,country,location,activity,injury,fatal_(y/n),time,species
15,Florida,4.0,9,9,9,9,9,8,9,9,5,9
17,Florida,6.0,8,8,8,8,8,8,8,8,7,8
19,Florida,8.0,8,8,8,8,8,8,8,8,7,8
20,Florida,9.0,5,5,5,5,5,5,5,5,4,5
61,Queensland,12.0,4,4,4,4,4,4,4,4,3,4
41,North Carolina,6.0,4,4,4,4,4,4,4,4,4,4
34,New South Wales,1.0,4,4,4,4,4,3,4,4,3,4
56,Queensland,1.0,4,3,4,4,3,4,4,3,3,4
14,Florida,3.0,3,3,3,3,3,3,3,3,3,3
40,New South Wales,12.0,3,3,3,3,3,3,3,3,2,3


# Unknown Shark

In [645]:
unknown_shark['fatal_(y/n)'].value_counts()

N          1204
Y           137
UNKNOWN       3
Name: fatal_(y/n), dtype: int64

In [646]:
unknown_shark['area'].value_counts()

Florida                309
New South Wales         94
Queensland              71
Hawaii                  64
KwaZulu-Natal           63
                      ... 
Saint-Beno�t             1
Trinidad                 1
Westmoreland Parish      1
Worcestershire           1
Inner Hebrides           1
Name: area, Length: 247, dtype: int64

In [647]:
unknown_shark.groupby(['area', 'month']).count().sort_values(['year'], ascending = False)

date  year  type  \
area                                               month                     
Florida                                            9.0      52    52    52   
                                                   8.0      37    37    37   
                                                   5.0      32    32    32   
                                                   10.0     31    31    31   
                                                   4.0      30    30    30   
...                                                        ...   ...   ...   
Kwajalein Atoll                                    1.0       1     1     1   
KwaZulu-Natal between Port Edward and Port St J... 6.0       1     1     1   
KwaZulu-Natal                                      6.0       1     1     1   
Kingston Parish                                    9.0       1     1     1   
Zadar County                                       7.0       1     1     1   

                                                          country  location  \
area                                               month                      
Florida                                            9.0         52        52   
                                                   8.0         37        37   
                                                   5.0         32        32   
                                                   10.0        31        31   
                                                   4.0         30        30   
...                                                           ...       ...   
Kwajalein Atoll                                    1.0          1         1   
KwaZulu-Natal between Port Edward and Port St J... 6.0          1         1   
KwaZulu-Natal                                      6.0          1         1   
Kingston Parish                                    9.0          1         1   
Zadar County                                       7.0          1         1   

                                                          activity  injury  \
area                                               month                     
Florida                                            9.0          52      52   
                                                   8.0          37      37   
                                                   5.0          31      32   
                                                   10.0         31      31   
                                                   4.0          29      30   
...                                                            ...     ...   
Kwajalein Atoll                                    1.0           1       1   
KwaZulu-Natal between Port Edward and Port St J... 6.0           1       1   
KwaZulu-Natal                                      6.0           1       1   
Kingston Parish                                    9.0           1       1   
Zadar County                                       7.0           1       1   

                                                          fatal_(y/n)  time  \
area                                               month                      
Florida                                            9.0             52    43   
                                                   8.0             37    30   
                                                   5.0             32    27   
                                                   10.0            31    25   
                                                   4.0             30    26   
...                                                               ...   ...   
Kwajalein Atoll                                    1.0              1     1   
KwaZulu-Natal between Port Edward and Port St J... 6.0              1     1   
KwaZulu-Natal                                      6.0              1     1   
Kingston Parish                                    9.0              1     0   
Zadar County                                       7

# Invalid Attacks

In [649]:
invalid_att['fatal_(y/n)'].value_counts()

N          13
Y           9
UNKNOWN     2
Name: fatal_(y/n), dtype: int64

In [655]:
invalid_att['injury'].value_counts()

No injury                                       19
FATAL                                           19
Probable drowning & scavenging                   7
Shark involvement prior to death unconfirmed     5
No details                                       5
                                                ..
Sharks fed on bodies of the drowned              1
Abdomen abraded                                  1
Thigh lacerated                                  1
Possible drowning / scavenging                   1
Human remains found in 4m, 900 kg shark          1
Name: injury, Length: 410, dtype: int64

In [651]:
invalid_att.groupby(['area', 'month']).count().sort_values(['year'], ascending = False)

date  year  type  country  location  activity  \
area               month                                                  
New South Wales    1.0      12    12    12       12        12        11   
Florida            8.0      11    11    11       11        11         9   
KwaZulu-Natal      1.0       9     9     9        9         9         4   
Florida            9.0       8     8     8        8         8         6   
                   10.0      8     8     8        8         8         6   
...                        ...   ...   ...      ...       ...       ...   
Ibaraki Prefecture 1.0       1     1     1        1         1         0   
Isle of Wight      8.0       1     1     1        1         1         1   
Kingston Parish    6.0       1     1     1        1         1         0   
KwaZulu-Natal      2.0       1     1     1        1         1         1   
Yucatan Channel    6.0       1     1     1        1         0         1   

                          injury  fatal_(y/n)  time  species  
area               month                                      
New South Wales    1.0        12            0     6       12  
Florida            8.0        11            1     6       11  
KwaZulu-Natal      1.0         9            0     2        9  
Florida            9.0         7            1     4        8  
                   10.0        8            1     5        8  
...                          ...          ...   ...      ...  
Ibaraki Prefecture 1.0         1            0     0        1  
Isle of Wight      8.0         1            0     0        1  
Kingston Parish    6.0         1            0     0        1  
KwaZulu-Natal      2.0         1            0     0        1  
Yucatan Channel    6.0         1            0     0        1  

[223 rows x 10 columns]

In [652]:
cshark['species'].value_counts()

UnknownShark                                                                                                                                                     1359
WhiteShark                                                                                                                                                        632
Invalid                                                                                                                                                           506
TigerShark                                                                                                                                                        281
BullShark                                                                                                                                                         180
NurseShark                                                                                                                                                         96
Blac